# Graph Class: no assumption on whether directed or undirected

### Graph parent class with following submethods:
### .vertices()
### .edges()
### .adjacent_vertices(vertex)
### .addvertex(vertex)
### .addedge(vertex1, vertex2) -- creates edge from vertex 1 to vertex2
### .removevertex(v)
### .removeedge(vertex1,vertex2)
### .BFS(startvertex) -- breadth first search from start vertex
### .distance(vertex1, vertex2) -- use BFS to measure distance between vertex1 and vertex2 (smallest # of edges between the two vertices)
### DFS(startvertex) -- depth first search from startvertex (recursive implementation)


In [6]:
#this defines the Graph parent class and has the most basic graph functionalities:
#list vertices and edges, add a vertex, and add an edge (firstvertex, secondvertex)
from queue import Queue
import numpy as np
import sys

class Graph:
    # if adjacency list dictionary not specified then assumes empty graph
    def __init__(self, G = None):
        if G == None:
            G = {}
        self._adjlist = G
        
    # loads adjacency list from text file. Must be formatted with first element of each line being
    # the start vertex and the rest being adjacent vertices with edges going to them fron start vertex.
    # text file must be parsed with each adjacent vertex being tab delimited.
    
    def loadfromfile(self, path):
        adjacencydict = {}
        with open(path, 'r') as f:
            for x in f:
                parsed_line = x.strip().split('\t')
                node_conn = {parsed_line[0]: parsed_line[1::]}
                adjacencydict.update(node_conn)      
        self._adjlist = adjacencydict
        
    
    def adjacencydict(self):
        return self._adjlist
    #lists all vertices in graph
    def vertices(self):
        adjdict = self._adjlist
        
        return set(adjdict)

    #lists all edges. edges are ordered tuples.
    
    def edges(self):
        edgelist = []
        adjdict = self._adjlist
        for vertex in adjdict:
            adjacentvertices = adjdict[vertex]
            for adjvertex in adjacentvertices:
                edgelist.append((vertex, adjvertex))
        return edgelist
    
    # returns vertices adjavent to vertex
    def adjacent_vertices(self, vertex):
        adjdict = self._adjlist
        return adjdict[vertex]
        
        

    # this just takes a single object in as a vertex
    def addvertex(self, vertex):
        adjdict = self._adjlist
        if vertex not in adjdict:
            adjdict[vertex] = []
    
   #this creates a single directed edge with a specified start and end vertex
   #start and end vertices have to already be in the graph
    
    def addedge(self, startvertex, endvertex):
        adjdict = self._adjlist
        if (startvertex in adjdict) and (endvertex in adjdict):
            adjdict[startvertex].append(endvertex)
        else:
            return
    
    
    def removevertex(self, vertex):
        if vertex in self.vertices():
            self._adjlist.pop(vertex)
            for vert in self._adjlist:
                if vertex in self._adjlist[vert]:
                    self._adjlist[vert].remove(vertex)
    
    
    def removeedge(self, vertex1, vertex2):
        vertlist = self.vertices()
        if (vertex1 in vertlist) and (vertex2 in vertlist):
            if vertex2 in self._adjlist[vertex1]:
                self._adjlist[vertex1].remove(vertex2)
        else:
            return
        
    
    #Breadth First Search from a specified start vertex. 
    def BFS(self, startvertex):
        adjdict = self._adjlist
        
        graphqueue = Queue(maxsize=0)
        graphqueue.put(startvertex)
        
        explored = [startvertex]
        
        while graphqueue.empty() == False:
            v = graphqueue.get_nowait() # dequeues element in graph element queue
            
            for element in adjdict[v]:
                if element not in explored:
                    explored.append(element) #now the element has been explored
                    graphqueue.put(element)        
        return explored
    
    #returns distance of shortest path between the start and end vertices.
    
    def distance(self, startvertex, endvertex):
        
        adjdict = self._adjlist
        
        graphqueue = Queue(maxsize=0)
        graphqueue.put(startvertex)
        explored = {startvertex: 0} #now changed to a dictionary to list level number
        
        
        while graphqueue.empty() == False:
            v = graphqueue.get_nowait() # dequeues element in graph element queue
            
            for element in adjdict[v]:
                if element not in explored:
                    explored.update({element: (explored[v]+1)}) #now the element has been explored, added to dictionary along with level number
                    graphqueue.put(element)    
                    
        if endvertex not in explored:
            return np.inf
        else:
            return explored[endvertex]
        
    #Depth First Search Implementation from a specified start vertex
    
    def DFS(self, vertex, explored = None):
        if explored == None:
            explored = [vertex]
            
        for adjvert in self._adjlist[vertex]:
            if adjvert not in explored:
                explored.append(adjvert)
                self.DFS(adjvert, explored)
        return explored 

    
    
        

# Undirected Graph Child Class

Each edge is bidirectional so there is no difference between (a,c) and (c,a). Want to build class that is reflective of that. Also want to create some specialized methods (undirected connectivity, etc)

### .edges() are now sets (takes priority over parent function)
### .addege(vertex1, vertex2) -- adds undirected edge (takes priority over parent function)
### .removeedge(vertex1, vertex2) -- removes undirected edge 
### .vertexdegree(vertex)  -- number of edges connected to vertex
### .connected_comp() -- gets connected components of undirected graph



In [7]:
class UndirectedGraph(Graph):
    
    def __init__(self, adjlist):
        super().__init__(adjlist)
    
    #order doesnt matter anymore so each edge should be represented as a set
    def edges(self):
        edgeset = {frozenset(x) for x in super().edges()}
        return edgeset
    
    # need to modify addedge and removeedge as ordering doesnt matter anymore
    
    def addedge(self, vertex1, vertex2):
        if vertex1 in super().vertices() and vertex2 in super().vertices():
            super().addedge(vertex1,vertex2)
            super().addedge(vertex2,vertex1)
        else:
            return
            
    
    def removeedge(self, vertex1, vertex2):
        if vertex1 in super().vertices() and vertex2 in super().vertices():
            super().removeedge(vertex1,vertex2)
            super().removeedge(vertex2,vertex1)
        else:
            return
           
    
    def vertexdegree(self, vertex):
        return len(self._adjlist[vertex])
    
    # define some private methods for internal class usage
    
    def connected_comp(self):
        explored = []
        connected = []
        connGraphs = []
        for node in super().vertices():
            if node not in explored:
                comp = super().BFS(node)
                explored.extend(comp)
                connected.append(comp)
        for networks in connected:
            x = UndirectedGraph({k: self._adjlist[k] for k in networks})
            connGraphs.append(x)
        return connGraphs
            
        

## Directed Graph Class
#### ReverseGraph() -- returns reversed graph
#### TopologicalSort() - DFS recursive implementation of topological sorting on a graph
#### KosarajuSCC() - returns strongly connected components (SCC) of a graph. Returns list of vertices in each SCC.

In [8]:
from copy import deepcopy

class DirectedGraph(Graph):
    
    def __init__(self, adjlist = None):
        super().__init__(adjlist)
        
# reverse Graph

    def ReverseGraph(self):
        adjdict = self._adjlist
        reverse_adjdict = {}
        for vertex in adjdict.keys():
            if vertex not in reverse_adjdict:
                reverse_adjdict.update({vertex: []})
                
            for node in adjdict[vertex]:
                if node not in reverse_adjdict:
                    reverse_adjdict.update({node: [vertex]})

                else:
                    if vertex not in reverse_adjdict[node]:
                        reverse_adjdict[node].append(vertex)

        return DirectedGraph(reverse_adjdict)
    
 # Topological Sort -- recursive DFS solution. Ouputs dictionary with label entries

    def TopologicalSort(self):
        seen = set()
        result= {}
        n = [len(self.vertices())]
        
        def DFS_indexed(node):
            seen.add(node)
            
            for neighbor in self.adjacent_vertices(node):
                if neighbor not in seen:
                    seen.add(neighbor)
                    DFS_indexed(neighbor)
            result.update({node: n[-1]})
            n[-1] = n[-1] - 1
            
        for vertex in self.vertices():
            if vertex not in seen:
                DFS_indexed(vertex)
        return result
        
# just converts topologically ordered vertex dictionary to an ordered list (in forward flow order -- last element is a sink of DAG)   
    def TopologicalOrderedList(self):
        top_dict = self.TopologicalSort()
        toplist = sorted(top_dict, key = lambda x: a[x])
        return toplist
    

    #Compute SCCS
    
    def KosarajuSCC(self):
    
        fintimes = [0]
        leaders = []
        ftimedict = {}
        
        seen = set()
        
        def DFSFirstPass(G, node):
            seen.add(node)
            
            for neighbor in G.adjacent_vertices(node):
                if neighbor not in seen:
                    seen.add(neighbor)
                    DFSFirstPass(G, neighbor)
            fintimes[-1] = fintimes[-1] + 1
            ftimedict.update({node: fintimes[-1]})
        
        def DFSSecondPass(G, node):
            
            seen.add(node)
            for neighbor in G.adjacent_vertices(node):
                if neighbor not in seen:
                    seen.add(neighbor)
                    DFSSecondPass(G, neighbor)
            
        
        #Reverse graph and first DFSloop to get finising times
        Grev = self.ReverseGraph()
        
        for vertex in Grev.vertices():
            if vertex not in seen:
                DFSFirstPass(Grev, vertex)
        
        #list of vertices ordered in terms of finishing times from largest to smallest
        
        verticesordered = sorted(ftimedict, key = lambda x: ftimedict[x] )[::-1]
        
        # second DFS loop time, reset seen, find the leaders
        seen = set()
        
        for vertex in verticesordered:
            if vertex not in seen:
                leaders.append(vertex)
                DFSSecondPass(self, vertex)
        
        
        
        SCCS = []
        G = deepcopy(self)
        # This goes over the leaders and plucks of all the SCCS by vertex:
        for node in leaders:
            x = G.DFS(node) 
            SCCS.append(x)
            for vertex in x:
                G.removevertex(vertex)
            
        return SCCS 
                
    
        
            

    

In [32]:
X = DirectedGraph({1:[4,10], 10: [], 4: [7], 7:[1], 9:[7,3], 3:[6], 6: [9], 8: [6,5], 2:[8], 5:[2]})

In [33]:
X.removevertex(10)

In [28]:
X.vertices()

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [34]:
X.edges()

[(1, 4),
 (4, 7),
 (7, 1),
 (9, 7),
 (9, 3),
 (3, 6),
 (6, 9),
 (8, 6),
 (8, 5),
 (2, 8),
 (5, 2)]

In [36]:
X.adjacencydict()

{1: [4], 4: [7], 7: [1], 9: [7, 3], 3: [6], 6: [9], 8: [6, 5], 2: [8], 5: [2]}

In [39]:
X.KosarajuSCC()

[[1, 4, 7], [9, 3, 6], [8, 5, 2]]